In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import os

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
print('Training Data avaiable is')
print(len(os.listdir('../input/skin-cancer-malignant-vs-benign/data/train/malignant')))

print('Testing Data avaiable is')
print(len(os.listdir('../input/skin-cancer-malignant-vs-benign/data/test/benign')))




Training Data avaiable is
1197
Testing Data avaiable is
360


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(228,228,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 226, 226, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 113, 113, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 111, 111, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 53, 53, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 43264)            

In [5]:
model.compile(
    loss='binary_crossentropy',
    optimizer = RMSprop(lr=0.0001),
    metrics=['acc']
)

In [6]:
path_training = '../input/skin-cancer-malignant-vs-benign/data/train/'
path_validation = '../input/skin-cancer-malignant-vs-benign/data/test/'
train_image_data_gen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest' 
)

train_image_generator=train_image_data_gen.flow_from_directory(
    path_training ,
    batch_size = 10,
    class_mode = 'binary' ,
    target_size = (228,228) ,
)

validation_image_data_gen=ImageDataGenerator(
    rescale=1./255
)
validation_image_generator= validation_image_data_gen.flow_from_directory(
    path_validation ,
    class_mode = 'binary' ,
    target_size = (228,228) ,
    batch_size = 10
)

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [ ]:
data = model.fit_generator(
    train_image_generator,
    validation_data=validation_image_generator,
    epochs=10,
    verbose=2
)

Epoch 1/10


In [ ]:
%matplotlib inline

acc=data.history['acc']
val_acc=data.history['val_acc']
loss=data.history['loss']
val_loss=data.history['val_loss']

epochs=range(len(acc)) 

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss ,'r','Training Loss')
plt.plot(epochs, val_loss, 'b','Validation Loss')
plt.title('Training and Validation Loss')
plt.figure()